# pygohome

## Python, Let's Go Home. Quickly.

*pygohome* is a 100% personal route optimizer in a known environment based on experience.

*You* walk/ride/drive frequently between known locations (home, work, school, shops, family, friends, …) using different routes, but would like to know the optimal route, that should take you the less time possible? *pygohome* uses your recorded GPX tracks to build a route network of *your* world with estimation on how long *you* need to get from A to B using the mean of transport of your choice.

## How it works

1. **Choose your mean of transport.** Bicycle works great, walking should too. Motorized vehicles may be served better by real-time online services.

2. **Track all your trips** using that mean of transport. Start the tracking before you leave, stop the tracking after you arrive. Ride/walk/drive normally, stop at lights, don't speed. OsmAnd works great (list of other apps needed). 1 or 2 seconds tracking interval is ok.

3. **Add POIs (and intersections) as waypoints.** Transfer all GPX files to your computer into a directory (OsmAnd names them `YYYY-MM-DD_HH-MM_DDD.gpx`). Load all of them into some GPX editor (JOSM works great).

4. **Create a new GPX file with waypoints** and add all points of interest (home, work, or any place where you started, deliberately paused or ended a trip) with an attribute `name=…`. Then add unnamed waypoints on each intersection where your tracks cross, split, or join. This is the biggest future plan in *pygohome* to offer a good editor with checks for consistency. Now you have to do it manually.

5. **Run the cell below** and an empty map will appear. All the action happens here from now on.

6. **Load your GPX files** with the tracks AND the file with waypoints. The current limit is 10MB for one upload, so if you have more data, do it in more batches or wait/contribute fix.

7. **Find the fastest route** where `src` and `dst` are the names of the POIs and `Quantile` is the a float number between `0.0` (take into account your best time on each road segment) and `1.0` (worst time), while `0.8` is considered a safe value.

In [1]:
from pygohome.world import World
import ipyleaflet as lf
import ipywidgets as wd


world = World()
m = lf.Map(
    zoom=14,
    center=(49.0, 8.4),
    interpolation="nearest",
    basemap=lf.basemaps.CartoDB.DarkMatter,
)


def action_load(change):
    for filename, content in sorted(uploader.value.items()):
        with out_box:
            print(f"Loading {filename}")
        if content["content"]:
            world.add_gpx(content["content"].decode("utf-8"))
    world._ensure_graph()
    pois = sorted(
        node
        for node in world.graph.nodes
        if isinstance(node, str) and not node.isdigit()
    )
    dropdown_src.options = pois
    dropdown_dst.options = pois


uploader = wd.FileUpload(accept=".gpx", multiple=True)
load_btn = wd.Button(description="Load GPX files")
load_btn.on_click(action_load)

init_box = wd.VBox(children=[uploader, load_btn])

route = lf.AntPath(locations=[])
m.add_layer(route)

dropdown_src = wd.Dropdown(options=[], description="Src:")
dropdown_dst = wd.Dropdown(options=[], description="Dst:")
float_slider = wd.FloatSlider(
    value=0.8,
    min=0,
    max=1.0,
    step=0.01,
    description="Quantile:",
    disabled=False,
    continuous_update=False,
    orientation="horizontal",
    readout=True,
    readout_format=".2f",
)
button = wd.Button(description="Find route")
changes = []


def action_route(change):
    fp = world.fastest_path(
        dropdown_src.value, dropdown_dst.value, quantile=float_slider.value
    )
    nodes = world.graph.nodes
    route.locations = [
        (nodes[node]["latitude"], nodes[node]["longitude"]) for node in fp.nodes
    ]
    m.center = (
        nodes[dropdown_src.value]["latitude"],
        nodes[dropdown_src.value]["longitude"],
    )


button.on_click(action_route)
route_box = wd.VBox(children=[dropdown_src, dropdown_dst, float_slider, button])


tab_box = wd.Tab(children=[init_box, route_box])
tab_box.set_title(0, "Load GPX files")
tab_box.set_title(1, "Fastest route")

out_box = wd.Output()

ctrl_box = wd.VBox(children=[tab_box, out_box])

ctrl_tab = lf.WidgetControl(widget=ctrl_box, position="topright")
m.add_control(ctrl_tab)

m